In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scraped_json//'
outputDir = '..//summarized_json//'
dataset = 'rivals'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dicti
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
rivalsData = mergeSourceFiles(dataset, inputDir, sourceFiles)

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
createNewID(idConfig[dataset], rivalsData, '_')
df = pandas.DataFrame(rivalsData)
df.head()

,school,year,playerName,city,state,highSchool,position,height,weight,stars,nationalRank,positionRank,stateRank,ID
0,michigan,2004,alanbranch,Albuquerque,NM,Cibola,OL,78.0,315.0,4,NaN,20.0,NaN,alanbranch_michigan
1,michigan,2009,craigroh,Scottsdale,AZ,Chaparral,DE,77.0,230.0,4,156.0,7.0,4.0,craigroh_michigan
2,maryland,2003,dremoore,Charlotte,NC,Independence,DT,76.0,275.0,2,NaN,NaN,NaN,dremoore_maryland
3,maryland,2016,darrylturner,Hyattsville,MD,DeMatha,ATH,69.0,188.0,4,NaN,17.0,7.0,darrylturner_maryland
4,rutgers,2005,ryanblaszczyk,Medford Lakes,NJ,Shawnee,OL,76.0,301.0,2,NaN,NaN,NaN,ryanblaszczyk_rutgers


In [ ]:
##With this - since I only want to keep 4 columns I'm going to create a new list of dicts
finalRivals = []
for record in rivalsData:
    finalRecord = {}
    finalRecord['ID'] = record['ID']
    finalRecord['rivals_stars'] = record['stars']
    finalRecord['rivals_natlRank'] = record['nationalRank']
    finalRecord['rivals_posRank'] = record['positionRank']
    finalRecord['rivals_stateRank'] = record['stateRank']
    finalRivals.append(finalRecord)
print(finalRivals)

[{'ID': 'alanbranch_michigan', 'rivals_stars': 4, 'rivals_natlRank': None, 'rivals_posRank': 20, 'rivals_stateRank': None}, {'ID': 'craigroh_michigan', 'rivals_stars': 4, 'rivals_natlRank': 156, 'rivals_posRank': 7, 'rivals_stateRank': 4}, {'ID': 'dremoore_maryland', 'rivals_stars': 2, 'rivals_natlRank': None, 'rivals_posRank': None, 'rivals_stateRank': None}, {'ID': 'darrylturner_maryland', 'rivals_stars': 4, 'rivals_natlRank': None, 'rivals_posRank': 17, 'rivals_stateRank': 7}, {'ID': 'ryanblaszczyk_rutgers', 'rivals_stars': 2, 'rivals_natlRank': None, 'rivals_posRank': None, 'rivals_stateRank': None}, {'ID': 'brandonware_pennstate', 'rivals_stars': 2, 'rivals_natlRank': None, 'rivals_posRank': None, 'rivals_stateRank': None}, {'ID': 'joeyelliott_purdue', 'rivals_stars': 3, 'rivals_natlRank': None, 'rivals_posRank': 18, 'rivals_stateRank': 5}, {'ID': 'devonmatthews_indiana', 'rivals_stars': 2, 'rivals_natlRank': None, 'rivals_posRank': None, 'rivals_stateRank': None}, {'ID': 'carlosh

In [ ]:
with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(finalRivals))